In [1]:
import pandas as pd
import json
import numpy as np
import pickle
import gc
from transformers import AutoConfig, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import random
import copy

In [2]:
### config
RANDOM_SEED = 42
SEED = 42
BATCH_SIZE = 64
NUM_WORKERS = 4
LR = 1e-4
EPOCHS = 10
WEIGHT_DECAY = 1e-4
device = 'cuda:0'

In [2]:
f = open('./u_transaction.json', 'r')
data = pd.read_json(f)
data

,10000000,10000001,10000002,10000003,10000004,10000005,10000006,10000007,10000008,10000009,...,10499990,10499991,10499992,10499993,10499994,10499995,10499996,10499997,10499998,10499999
12,"[16, 0, 0, 14, 16, 0, 0, 15, 16, 0, 0, 0, 0, 1...","[0, 0, 0, 0, 14, 0, 14, 0, 16, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 13, 0, 0, 0, 0, 0, 0, 0, 16, 15, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0,...","[0, 15, 0, 0, 0, 0, 0, 0, 13, 0, 16, 16, 14, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10,"[15, 0, 16, 13, 15, 16, 16, 16, 15, 16, 16, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[14, 0, 0, 16, 0, 0, 14, 13, 0, 0, 0, 0, 0, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[16, 0, 0, 0, 16, 0, 16, 0, 15, 15, 15, 0, 0, ...","[0, 0, 14, 0, 0, 0, 0, 16, 13, 0, 0, 15, 0, 16...","[0, 0, 0, 13, 0, 15, 13, 0, 15, 14, 14, 0, 11,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[16, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 13, 0, 16, 15, 0, 0, 0, 14, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 16, 15, 16, 0, 0, 16, 0, 0, 0, 16, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 13, 0, 12, 0, 0, 0, 0, 0, 0, 0, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
# t = []
# print(data[10000000].keys())
# for item in data[10000000]:
#     t.append(item)

# tmp = np.array(t).transpose().tolist()
# for itr1 in range(len(tmp)):
#     for itr2 in range(len(tmp[0])):
#         if tmp[itr1][itr2] != 0:
#             tmp[itr1][itr2] = list(data[10000000].keys())[itr2]
# tmp

In [4]:
data_dict = {}
for itr1 in data.columns:
    t = []
    for item in data[itr1]:
        t.append(item)
    # tmp = np.array(t).transpose().tolist()
    # print(tmp)
    # for itr2 in range(len(tmp)):
    #     for itr3 in range(len(tmp[itr2])):
    #         if tmp[itr2][itr3] != 0:
    #             tmp[itr2][itr3] = list(data[itr1].keys())[itr3]
    # data_dict[str(itr1)] = tmp
    data_dict[str(itr1)] = t
    # print(data_dict[str(itr1)])
# data_dict

In [ ]:
# save the transformed data into another file
# f = open('./data_transaction_tag.pickle', 'wb')
# pickle.dump(data_dict, f)
# f.close()

In [ ]:
del data
gc.collect()

0

In [ ]:
data_dict['10000000']

[[16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [14, 13, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0],
 [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [15, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 

In [ ]:
# config = AutoConfig.from_pretrained('bert-base-uncased')
# model =  AutoModel.from_config(config)
# encoder_layer = torch.nn.TransformerEncoderLayer(d_model=16, nhead=4, dim_feedforward=64, dropout=0.1, activation='relu')
# model = torch.nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=4)

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.encoder = torch.nn.TransformerEncoder(
            encoder_layer=torch.nn.TransformerEncoderLayer(d_model=16, nhead=4, dim_feedforward=64, dropout=0.1, activation='relu'),
            num_layers=6)


    def forward(self, x):
        x = self.encoder(x)
        return x


def collate_function(data):
    for itr1 in range(len(data)):
        for itr2 in range(len(data[itr1][0])):
            data[itr1][0][itr2] /= 16
            data[itr1][1][itr2] /= 16
    return data

In [ ]:
print(torch.unsqueeze(torch.Tensor(data_dict['10000000'][0]), 0))
# print(torch.unsqueeze(torch.Tensor(data_dict['10000000'][0]), 0))
data = torch.unsqueeze(torch.Tensor(data_dict['10000000']), 0)
print(len(data[0]))
model(data)
# print(model(data))
# print(len(torch.Tensor(data_dict['10000000'])))
softmax = torch.nn.Softmax(dim=1)
print(softmax(model(data)))

tensor([[16., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.]])
24
tensor([[[0.0432, 0.0422, 0.0434, 0.0356, 0.0318, 0.0248, 0.0289, 0.0413,
          0.0507, 0.0506, 0.0318, 0.0315, 0.0596, 0.0342, 0.0331, 0.0348],
         [0.0367, 0.0267, 0.0637, 0.0390, 0.0184, 0.0264, 0.0289, 0.0462,
          0.0365, 0.0805, 0.0289, 0.0466, 0.1007, 0.0363, 0.0268, 0.0261],
         [0.0218, 0.0820, 0.0282, 0.0330, 0.0318, 0.0279, 0.0503, 0.0294,
          0.0285, 0.0339, 0.0556, 0.0808, 0.0228, 0.0320, 0.0637, 0.0360],
         [0.0470, 0.0423, 0.0415, 0.0278, 0.0780, 0.0279, 0.0521, 0.0396,
          0.0344, 0.0213, 0.0495, 0.0145, 0.0334, 0.0507, 0.0320, 0.0541],
         [0.0501, 0.0434, 0.0393, 0.0307, 0.0939, 0.0407, 0.0411, 0.0519,
          0.0412, 0.0273, 0.0395, 0.0233, 0.0162, 0.0337, 0.0395, 0.0334],
         [0.0419, 0.0423, 0.0215, 0.0250, 0.0437, 0.0509, 0.0557, 0.0371,
          0.0397, 0.0214, 0.0562, 0.0499, 0.0257, 0.0331, 0.0411, 0.0436],
   

In [ ]:
m = torch.nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output

tensor([[0.4268, 0.1887, 0.3845],
        [0.2107, 0.0442, 0.7450]])

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.sequence = [torch.Tensor(s[0]) for s in data]
        self.targets = [torch.Tensor(s[1]) for s in data]
        self.mask = [(s[2]) for s in data]

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, idx):
        return self.sequence[idx], self.targets[idx], self.mask[idx]

In [5]:
# This block is for random mask
dataset_dict = copy.deepcopy(data_dict)
dataset_masked_position = {}
for transaction_id in dataset_dict.keys():
    random_list = []
    while(len(random_list) < 3):
        tmp = random.randint(0,23)
        if tmp not in random_list:
            random_list.append(tmp)
    dataset_masked_position[str(transaction_id)] = random_list
    for itr1 in range(len(random_list)):
        for itr2 in range(len(dataset_dict[transaction_id][random_list[itr1]])):
            dataset_dict[transaction_id][random_list[itr1]][itr2] = -1

IndexError: list index out of range

In [6]:
next_year_predict_dataset = {}
next_year_predict_dataset_target = {}
# create dataset for linear
for usid in data_dict.keys():
    tmp = copy.deepcopy(data_dict[str(usid)])
    tmp_target = []
    for itr1 in range(len(tmp)):
        tmp_target.append([tmp[itr1][-1]])
        tmp[itr1] = tmp[itr1][0:-1]
    next_year_predict_dataset[str(usid)] = tmp
    next_year_predict_dataset_target[str(usid)] = tmp_target

In [9]:
# data_2_dict = copy.deepcopy(data_dict)
print(next_year_predict_dataset['10000000'])
print(next_year_predict_dataset_target['10000000'])

KeyError: '10000000'

In [7]:
f = open('./dataset_next_month.pickle', 'wb')
pickle.dump(next_year_predict_dataset, f)
f.close()
f = open('./dataset_next_month_target.pickle', 'wb')
pickle.dump(next_year_predict_dataset_target, f)
f.close()

In [ ]:
print(dataset_dict['10000000'])
print(dataset_masked_position['10000000'])
print(data_dict['10000000'])

[[16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [14, 13, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0], [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [15, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# data_list = []
# for itr1 in dataset_dict.keys():
#     data_list.append((dataset_dict[str(itr1)], data_dict[str(itr1)], dataset_masked_position[str(itr1)]))
print((data_list[0]))
dataset = MyDataset(data_list)
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, num_workers = NUM_WORKERS, collate_fn = collate_function)

([[16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [14, 13, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0], [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [15, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# for data, target, mask in dataset:
#     print(data)
#     print(target)
#     print(mask)
#     break
# for data in dataloader:
#     print(data)
#     break


[(tensor([[ 0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 15.,  0.,  0.,
         16.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         16.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1.],
        

In [2]:
f = open('./dataset_next_month.pickle', 'rb')
data = pickle.load(f)
f.close()

In [7]:
from tqdm import tqdm
print(len(data['10000000'][0]))

for i in tqdm(data.keys()):
    for itr in range(len(data[i])):
        data[i][itr] = data[i][itr][12:]

print(len(data['10000000'][0]))




23


100%|██████████| 498040/498040 [00:02<00:00, 218913.40it/s]

11


In [ ]:
f = open('./dataset_next_month_12.pickle', 'wb')
pickle.dump(data, f)
f.close()